
# Εισαγωγή στα Bayesian Networks – Student Network

Σε αυτό το notebook θα δούμε ένα κλασικό παράδειγμα **Bayesian Network**
(το *Student network*) και θα εκτελέσουμε βασικά ερωτήματα inference.

## Στόχοι

- Να ορίσουμε τη δομή (DAG) ενός Bayesian Network με `pgmpy`.
- Να ορίσουμε τους πίνακες πιθανοτήτων (CPDs) για κάθε κόμβο.
- Να χρησιμοποιήσουμε τον αλγόριθμο Variable Elimination για inference.
- Να απεικονίσουμε εποπτικά κάποιες posterior πιθανότητες.



## Θεωρία: Bayesian Networks

Ένα **Bayesian Network** είναι ένα κατευθυνόμενο ακύκλο γράφημα (DAG)
όπου κάθε κόμβος αντιστοιχεί σε μια τυχαία μεταβλητή και κάθε τόξο
εκφράζει μία άμεση πιθανολογική εξάρτηση.

Αν έχουμε μεταβλητές $X_1, \dots, X_n$ και για κάθε $X_i$ το σύνολο
γονέων $\mathrm{Pa}(X_i)$ στο γράφημα, τότε η κοινή κατανομή γράφεται:

$$
P(X_1, \dots, X_n) = \prod_i P\bigl(X_i \mid \mathrm{Pa}(X_i)\bigr)
$$

Η τοπική δομή του γράφου κωδικοποιεί conditional independencies
ανάμεσα στις μεταβλητές, γεγονός που επιτρέπει πιο αποδοτικό
inference σε σχέση με την πλήρη κοινή κατανομή.



## Το Student network

Χρησιμοποιούμε το κλασικό παράδειγμα *Student* με τις μεταβλητές:

- `Difficulty` ∈ {`easy`, `hard`}
- `Intelligence` ∈ {`low`, `high`}
- `Grade` ∈ {`A`, `B`, `C`}
- `SAT` ∈ {`low`, `high`}
- `Letter` ∈ {`weak`, `strong`}

Δομή:

- `Difficulty → Grade`
- `Intelligence → Grade`
- `Intelligence → SAT`
- `Grade → Letter`

Η κοινή κατανομή παραγοντοποιείται ως:

$$
P(D, I, G, S, L) = P(D) P(I) P(G \mid D, I) P(S \mid I) P(L \mid G)
$$

όπου:

- $D$ = Difficulty,
- $I$ = Intelligence,
- $G$ = Grade,
- $S$ = SAT,
- $L$ = Letter.


In [ ]:

# Imports για Bayesian Networks με pgmpy

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

import matplotlib.pyplot as plt


In [ ]:

# Ορίζουμε τη δομή (DAG) του Student Network

model = BayesianModel(
    [
        ("Difficulty", "Grade"),
        ("Intelligence", "Grade"),
        ("Intelligence", "SAT"),
        ("Grade", "Letter"),
    ]
)

# Ορίζουμε τα CPDs (πιθανότητες είναι διδακτικές, όχι πραγματικά δεδομένα)

cpd_difficulty = TabularCPD(
    variable="Difficulty",
    variable_card=2,
    values=[[0.6], [0.4]],
    state_names={"Difficulty": ["easy", "hard"]},
)

cpd_intelligence = TabularCPD(
    variable="Intelligence",
    variable_card=2,
    values=[[0.7], [0.3]],
    state_names={"Intelligence": ["low", "high"]},
)

cpd_grade = TabularCPD(
    variable="Grade",
    variable_card=3,
    values=[
        [0.30, 0.05, 0.90, 0.50],  # P(G=A | I,D)
        [0.40, 0.25, 0.08, 0.30],  # P(G=B | I,D)
        [0.30, 0.70, 0.02, 0.20],  # P(G=C | I,D)
    ],
    evidence=["Intelligence", "Difficulty"],
    evidence_card=[2, 2],
    state_names={
        "Grade": ["A", "B", "C"],
        "Intelligence": ["low", "high"],
        "Difficulty": ["easy", "hard"],
    },
)

cpd_sat = TabularCPD(
    variable="SAT",
    variable_card=2,
    values=[
        [0.95, 0.20],  # P(SAT=low  | I)
        [0.05, 0.80],  # P(SAT=high | I)
    ],
    evidence=["Intelligence"],
    evidence_card=[2],
    state_names={
        "SAT": ["low", "high"],
        "Intelligence": ["low", "high"],
    },
)

cpd_letter = TabularCPD(
    variable="Letter",
    variable_card=2,
    values=[
        [0.10, 0.40, 0.90],  # P(L=weak   | G)
        [0.90, 0.60, 0.10],  # P(L=strong | G)
    ],
    evidence=["Grade"],
    evidence_card=[3],
    state_names={
        "Letter": ["weak", "strong"],
        "Grade": ["A", "B", "C"],
    },
)

model.add_cpds(
    cpd_difficulty,
    cpd_intelligence,
    cpd_grade,
    cpd_sat,
    cpd_letter,
)

# Έλεγχος ορθότητας του μοντέλου
model.check_model()

print("Κόμβοι:", list(model.nodes()))
print("Τόξα:", list(model.edges()))


In [ ]:

# Εμφανίζουμε τα CPDs για να τα δούμε/σχολιάσουμε

for cpd in model.get_cpds():
    print(cpd)
    print()


In [ ]:

# Ορίζουμε έναν inference engine (Variable Elimination)

infer = VariableElimination(model)

# Παράδειγμα 1: P(Grade | Difficulty=hard, Intelligence=high)
q1 = infer.query(
    variables=["Grade"],
    evidence={"Difficulty": "hard", "Intelligence": "high"},
    show_progress=False,
)
print("P(Grade | Difficulty=hard, Intelligence=high):")
print(q1)
print()

# Παράδειγμα 2: P(Intelligence | Grade=A, SAT=high)
q2 = infer.query(
    variables=["Intelligence"],
    evidence={"Grade": "A", "SAT": "high"},
    show_progress=False,
)
print("P(Intelligence | Grade=A, SAT=high):")
print(q2)
print()

# Παράδειγμα 3: P(Intelligence | Letter=strong)
q3 = infer.query(
    variables=["Intelligence"],
    evidence={"Letter": "strong"},
    show_progress=False,
)
print("P(Intelligence | Letter=strong):")
print(q3)


In [ ]:

# Εποπτική απεικόνιση: πώς αλλάζει η κατανομή του Intelligence
# όταν προσθέτουμε evidence στο Letter.

# 1) Prior P(Intelligence)
prior_I = infer.query(variables=["Intelligence"], show_progress=False)

# 2) Posterior P(Intelligence | Letter=weak)
post_I_weak = infer.query(
    variables=["Intelligence"],
    evidence={"Letter": "weak"},
    show_progress=False,
)

# 3) Posterior P(Intelligence | Letter=strong)
post_I_strong = infer.query(
    variables=["Intelligence"],
    evidence={"Letter": "strong"},
    show_progress=False,
)

states = ["low", "high"]

prior_vals = [prior_I.values[0], prior_I.values[1]]
weak_vals = [post_I_weak.values[0], post_I_weak.values[1]]
strong_vals = [post_I_strong.values[0], post_I_strong.values[1]]

# Για απλότητα, κάνουμε τρία ξεχωριστά bar plots

fig, ax = plt.subplots()
ax.bar(states, prior_vals)
ax.set_title("P(Intelligence) (prior)")
ax.set_ylabel("Πιθανότητα")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots()
ax.bar(states, weak_vals)
ax.set_title("P(Intelligence | Letter=weak)")
ax.set_ylabel("Πιθανότητα")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots()
ax.bar(states, strong_vals)
ax.set_title("P(Intelligence | Letter=strong)")
ax.set_ylabel("Πιθανότητα")
fig.tight_layout()
plt.show()
